In [21]:
import nrrd
import os
import pandas as pd
import numpy as np
from scipy.ndimage import zoom
import csv
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Conv3DTranspose, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model, save_model

In [22]:
data_path = 'F:\\Birva_Harsh\\New_nrrd'

In [23]:
ct_paths = []

for file in os.listdir(data_path):
    
    if file.endswith('.nrrd'):
        ct_path = os.path.join(data_path, file)
        ct_paths.append(ct_path)

In [24]:
csv_paths = [path.replace('.nrrd', '.csv') for path in ct_paths]

In [25]:
def read_csv_to_dict_LM_INFO(csv_file):
    LM_INFO = {}
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)       
        for rows in reader:
            LM_INFO[rows[0]] = np.asarray([float(rows[1]), float(rows[2]), float(rows[3])])
            #print(rows[0], ':', float(rows[1]), float(rows[2]), float(rows[3]))
            
    return LM_INFO

In [26]:
def zoom_1mm(img,original_space,new_space=[1.0,1.0,1.0]):
    zoom_factor = [original_space[i] / new_space[i] for i in range(3)]
    zoomed_image = zoom(img, zoom_factor, order=0)
    
    return zoomed_image

In [27]:
def knee_centre(path, original_space):
    KC = read_csv_to_dict_LM_INFO(path)['FKC']
    KC_1mm = [round(original_space[i] * KC[i]) for i in range(3)]
    
    return KC_1mm

In [28]:
def CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(img, centered_at, crop_size):
    crop_size = np.asanyarray(crop_size)
    img = np.pad(img, ((128,128),(128,128),(128,128)),'edge')   

    sp_idx = np.asarray(128+centered_at-crop_size//2,dtype=int)
    ep_idx = np.asarray(128+centered_at+crop_size//2,dtype=int)
       
    return img[sp_idx[0]:ep_idx[0], sp_idx[1]:ep_idx[1], sp_idx[2]:ep_idx[2]]

In [33]:
def crop_KC(ct_paths, csv_paths, output_path):
    for i in range(len(ct_paths)):
        img, info = nrrd.read(ct_paths[i])
        spc_dir = np.diag(info['space directions'])
    
        img_1mm = zoom_1mm(img, spc_dir)
        KC = knee_centre(csv_paths[i], spc_dir)
        
        info['sizes'] = np.array([256.0,256.0,256.0])
        info['space directions'] = np.eye(3)

        crop_ct = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(img_1mm, np.array(KC), [256,256,256])

        output_img_path = os.path.join(output_path, os.path.basename(ct_paths[i]))
        nrrd.write(output_img_path, crop_ct, info)
        print(i)

In [34]:
crop_KC(np.array(ct_paths),np.array(csv_paths),f'F:\\Birva_Harsh\\New_nrrd_crop')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
